In [9]:
# Lab 7 Learning rate and Evaluation
from functools import partial as bind
import numpy as np
import tensorflow as tf
assert(tf.__version__.find('2') == 0)
tf.random.set_seed(777)  # for reproducibility

x_data = tf.Variable([[1, 2, 1],
          [1, 3, 2],
          [1, 3, 4],
          [1, 5, 5],
          [1, 7, 5],
          [1, 2, 5],
          [1, 6, 6],
          [1, 7, 7]], dtype=tf.float32)
y_data = tf.Variable([[0, 0, 1],
          [0, 0, 1],
          [0, 0, 1],
          [0, 1, 0],
          [0, 1, 0],
          [0, 1, 0],
          [1, 0, 0],
          [1, 0, 0]], dtype=tf.float32)

# Evaluation our model using this test dataset
x_test = tf.Variable([[2, 1, 1],
          [3, 1, 2],
          [3, 3, 4]], dtype=tf.float32)
y_test = tf.Variable([[0, 0, 1],
          [0, 0, 1],
          [0, 0, 1]], dtype=tf.float32)

# Try to change learning_rate to small numbers
for lr in [1.5, 1e-10, 0.1]:
    W = tf.Variable(tf.random.normal([3, 3]))
    b = tf.Variable(tf.random.normal([3]))

    # tf.nn.softmax computes softmax activations
    # softmax = exp(logits) / reduce_sum(exp(logits), dim)
    hypothesis = lambda X: tf.nn.softmax(tf.matmul(X, W) + b)

    # Cross entropy cost/loss
    @tf.function
    def cost(X, Y):
        return tf.reduce_mean(-tf.reduce_sum(Y * tf.math.log(hypothesis(X)), axis=1))

    tf.print("When lr:", lr)
    optimizer = tf.keras.optimizers.SGD(learning_rate=lr)

    # Correct prediction Test model
    prediction = lambda X: tf.argmax(hypothesis(X), 1)
    is_correct = lambda X, Y: tf.equal(prediction(X), tf.argmax(Y, 1))
    accuracy = lambda X, Y: tf.reduce_mean(tf.cast(is_correct(X, Y), tf.float32))

    # Launch graph
    for step in range(201):
        # Minimize
        optimizer.minimize(bind(cost, x_data, y_data), var_list=[W, b])

        if step % 20 == 0:
            # evaluate training accuracy
            tf.print("Step: ", step, "cost: ", bind(cost, x_data, y_data)(), "W: ", W)

    # predict
    tf.print("Prediction:", prediction(x_test))
    # Calculate the accuracy
    tf.print("Accuracy: ", accuracy(x_test, y_test))
    tf.print("-------------------------------------------------")

'''
when lr = 1.5

0 5.73203 [[-0.30548954  1.22985029 -0.66033536]
 [-4.39069986  2.29670858  2.99386835]
 [-3.34510708  2.09743214 -0.80419564]]
1 23.1494 [[ 0.06951046  0.29449689 -0.0999819 ]
 [-1.95319986 -1.63627958  4.48935604]
 [-0.90760708 -1.65020132  0.50593793]]
2 27.2798 [[ 0.44451016  0.85699677 -1.03748143]
 [ 0.48429942  0.98872018 -0.57314301]
 [ 1.52989244  1.16229868 -4.74406147]]
3 8.668 [[ 0.12396193  0.61504567 -0.47498202]
 [ 0.22003263 -0.2470119   0.9268558 ]
 [ 0.96035379  0.41933775 -3.43156195]]
4 5.77111 [[-0.9524312   1.13037777  0.08607888]
 [-3.78651619  2.26245379  2.42393875]
 [-3.07170963  3.14037919 -2.12054014]]
5 inf [[ nan  nan  nan]
 [ nan  nan  nan]
 [ nan  nan  nan]]
6 nan [[ nan  nan  nan]
 [ nan  nan  nan]
 [ nan  nan  nan]]
 ...
Prediction: [0 0 0]
Accuracy:  0.0
-------------------------------------------------
When lr = 1e-10

0 5.73203 [[ 0.80269563  0.67861295 -1.21728313]
 [-0.3051686  -0.3032113   1.50825703]
 [ 0.75722361 -0.7008909  -2.10820389]]
1 5.73203 [[ 0.80269563  0.67861295 -1.21728313]
 [-0.3051686  -0.3032113   1.50825703]
 [ 0.75722361 -0.7008909  -2.10820389]]
...
199 5.73203 [[ 0.80269563  0.67861295 -1.21728313]
 [-0.3051686  -0.3032113   1.50825703]
 [ 0.75722361 -0.7008909  -2.10820389]]
200 5.73203 [[ 0.80269563  0.67861295 -1.21728313]
 [-0.3051686  -0.3032113   1.50825703]
 [ 0.75722361 -0.7008909  -2.10820389]]
Prediction: [0 0 0]
Accuracy:  0.0
-------------------------------------------------
When lr = 0.1

0 5.73203 [[ 0.72881663  0.71536207 -1.18015325]
 [-0.57753736 -0.12988332  1.60729778]
 [ 0.48373488 -0.51433605 -2.02127004]]
1 3.318 [[ 0.66219079  0.74796319 -1.14612854]
 [-0.81948912  0.03000021  1.68936598]
 [ 0.23214608 -0.33772916 -1.94628811]]
...
199 0.672261 [[-1.15377033  0.28146935  1.13632679]
 [ 0.37484586  0.18958236  0.33544877]
 [-0.35609841 -0.43973011 -1.25604188]]
200 0.670909 [[-1.15885413  0.28058422  1.14229572]
 [ 0.37609792  0.19073224  0.33304682]
 [-0.35536593 -0.44033223 -1.2561723 ]]
Prediction: [2 2 2]
Accuracy:  1.0
'''


When lr: 1.5
Step:  0 cost:  12.9678288 W:  [[1.14470661 0.93543148 -0.991899252]
 [2.43914 2.03430772 -3.48942089]
 [2.68254757 2.71287251 -3.80668187]]
Step:  20 cost:  nan W:  [[-nan(ind) -nan(ind) -nan(ind)]
 [-nan(ind) -nan(ind) -nan(ind)]
 [-nan(ind) -nan(ind) -nan(ind)]]
Step:  40 cost:  nan W:  [[-nan(ind) -nan(ind) -nan(ind)]
 [-nan(ind) -nan(ind) -nan(ind)]
 [-nan(ind) -nan(ind) -nan(ind)]]
Step:  60 cost:  nan W:  [[-nan(ind) -nan(ind) -nan(ind)]
 [-nan(ind) -nan(ind) -nan(ind)]
 [-nan(ind) -nan(ind) -nan(ind)]]
Step:  80 cost:  nan W:  [[-nan(ind) -nan(ind) -nan(ind)]
 [-nan(ind) -nan(ind) -nan(ind)]
 [-nan(ind) -nan(ind) -nan(ind)]]
Step:  100 cost:  nan W:  [[-nan(ind) -nan(ind) -nan(ind)]
 [-nan(ind) -nan(ind) -nan(ind)]
 [-nan(ind) -nan(ind) -nan(ind)]]
Step:  120 cost:  nan W:  [[-nan(ind) -nan(ind) -nan(ind)]
 [-nan(ind) -nan(ind) -nan(ind)]
 [-nan(ind) -nan(ind) -nan(ind)]]
Step:  140 cost:  nan W:  [[-nan(ind) -nan(ind) -nan(ind)]
 [-nan(ind) -nan(ind) -nan(ind)]
 [

'\nwhen lr = 1.5\n\n0 5.73203 [[-0.30548954  1.22985029 -0.66033536]\n [-4.39069986  2.29670858  2.99386835]\n [-3.34510708  2.09743214 -0.80419564]]\n1 23.1494 [[ 0.06951046  0.29449689 -0.0999819 ]\n [-1.95319986 -1.63627958  4.48935604]\n [-0.90760708 -1.65020132  0.50593793]]\n2 27.2798 [[ 0.44451016  0.85699677 -1.03748143]\n [ 0.48429942  0.98872018 -0.57314301]\n [ 1.52989244  1.16229868 -4.74406147]]\n3 8.668 [[ 0.12396193  0.61504567 -0.47498202]\n [ 0.22003263 -0.2470119   0.9268558 ]\n [ 0.96035379  0.41933775 -3.43156195]]\n4 5.77111 [[-0.9524312   1.13037777  0.08607888]\n [-3.78651619  2.26245379  2.42393875]\n [-3.07170963  3.14037919 -2.12054014]]\n5 inf [[ nan  nan  nan]\n [ nan  nan  nan]\n [ nan  nan  nan]]\n6 nan [[ nan  nan  nan]\n [ nan  nan  nan]\n [ nan  nan  nan]]\n ...\nPrediction: [0 0 0]\nAccuracy:  0.0\n-------------------------------------------------\nWhen lr = 1e-10\n\n0 5.73203 [[ 0.80269563  0.67861295 -1.21728313]\n [-0.3051686  -0.3032113   1.5082570